In [1]:
from utils import *

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


## Analyzing Neuron Results
- Just play around but make it easy for changing what to evaluate
- Don't read version "". num_neurons = 2024. This could crash the run

### Results
- (vesion 2 neurons 20 approx)
    - Layer 1 und Layer 6 haben sehr starke Ergebnisse
    - Ergebnisse werden besser mit position
    - Layer 4 und 5 sind sehr stark im rim
    - final sieht nicht gut aus haha (L4 besser als andere)
- (vesion 2 neurons 20 real)
    - layer 0 und 7 sind plötzlich gut
    - bisschen besser halt aber relativ ähnlich
- (version 2 neuron 2048 real)
    - relativ ähnlich actually
- Das war vielleicht nicht klug das mit dem Threshold zu machen ... dadurch ... ohne threshold wäre sehr interessant gewesen ...
Was ist Fazit??
- also wenn ich eins mit richtig vielen Neurons hätte, hätte ich halt sagen können so bro wenn wir alle Dinger nehmen dann wird das cool (wobei das vielleicht auch mit dem max garnicht hin haut)
- Mein Fazit ist soo, es gibt neurons die den Regeln folgen, und aber letztendlich sind entweder viele Neurons beteiligt oder ich habe nicht die richtigen gefunden.
- Ich sollte eine baselnine machen. Was ist die accuracy wenn wir nur + flipped predicten z.B. oder bei final wenn wir random predicten ...


In [53]:
def get_results(directory):
    mask_sum_path = "flipped_circuit_mask_sum.pth"
    neuron_acts_dict_path = "flipped_circuit_neuron_acts_diff_dict.pth"
    circuit_results_dict_path = "flipped_circuit_results_dict.pth"

    mask_sum = t.load(os.path.join(directory, mask_sum_path))
    if not os.path.exists(os.path.join(directory, neuron_acts_dict_path)):
        neuron_acts_dict = None
    else:
        neuron_acts_dict = t.load(os.path.join(directory, neuron_acts_dict_path))
    results_dict = t.load(os.path.join(directory, circuit_results_dict_path))
    rule_max = 1035
    return mask_sum, neuron_acts_dict, results_dict, rule_max

version = "_actually_correct"
# version = ""
num_neurons = 2048
# num neurons = 2048
real_attention = True
# real_neuron_acts = False
real_neuron_acts = False
directory = f"flipping_circuit_results/num_neurons_{num_neurons}_real_attention_{real_attention}_real_neuron_acts_{real_neuron_acts}{version}"
mask_sum_neuron_acts_approx, neuron_acts_dict_neuron_acts_approx, results_dict_neuron_acts_approx, rule_max = get_results(directory)
if version == "_3" or version == "_actually_correct":
    mask_normal_sum_neuron_acts_approx = t.load(f"flipping_circuit_results/num_neurons_{num_neurons}_real_attention_{real_attention}_real_neuron_acts_{real_neuron_acts}{version}/flipped_circuit_mask_normal_sum.pth")
real_neuron_acts = True
directory = f"flipping_circuit_results/num_neurons_{num_neurons}_real_attention_{real_attention}_real_neuron_acts_{real_neuron_acts}{version}"
mask_sum_neuron_acts_real, neuron_acts_dict_neuron_acts_real, results_dict_neuron_acts_real, rule_max = get_results(directory)
directory = f"flipping_circuit_results/baseline_{10}"
mask_sum_base_flipped, _, results_dict_base_flipped, rule_max = get_results(directory)
directory = f"flipping_circuit_results/baseline_{-10}"
mask_sum_base_not_flipped, _, results_dict_base_not_flipped, rule_max = get_results(directory)
directory = f"flipping_circuit_results/baseline_{0}"
mask_sum_base, _, results_dict_base, rule_max = get_results(directory)
if version == "_3" or version == "_actually_correct":
    mask_normal_sum_neuron_acts_real = t.load(f"flipping_circuit_results/num_neurons_{num_neurons}_real_attention_{real_attention}_real_neuron_acts_{real_neuron_acts}{version}/flipped_circuit_mask_normal_sum.pth")
    mask_normal_sum_base_flipped = t.load(f"flipping_circuit_results/baseline_{10}/flipped_circuit_mask_normal_sum.pth")
    mask_normal_sum_base_not_flipped = t.load(f"flipping_circuit_results/baseline_{-10}/flipped_circuit_mask_normal_sum.pth")
    mask_normal_sum_base = t.load(f"flipping_circuit_results/baseline_{0}/flipped_circuit_mask_normal_sum.pth")


In [54]:
avg_neuron_count = results_dict_neuron_acts_approx["avg_neuron_count"]
count_not_nan_per_layer = (avg_neuron_count == avg_neuron_count).sum(dim=-1)
avg_neuron_count[avg_neuron_count != avg_neuron_count] = 0
avg_neuron_count = avg_neuron_count.sum(dim=-1) / count_not_nan_per_layer
# Neuron acts approx acc: 0.13220436871051788
# Neuron acts real acc: 0.15360864996910095
# Avg Neuron Counts: tensor([148.3725,  20.5081,  48.2444,  85.6759, 199.0628, 291.2837, 447.8264, 97.4095], device='cuda:0')
avg_neuron_count

tensor([148.3725,  20.5081,  48.2444,  85.6759, 199.0628, 291.2837, 447.8264,
         97.4095], device='cuda:0')

In [56]:
def get_accuracy(results_dict, mask_sum, diff_or_final="diff", sum_dims = []):
    # Diff views positive logits as a positive sample, even when the approximation did not predict a change ...
    # Final views positive logits as a positive sample, only when the approximation predicted a change ...
    tp_str = f"TP_{diff_or_final}"
    fp_str = f"FP_{diff_or_final}"
    tn_str = f"TN_{diff_or_final}"
    fn_str = f"FN_{diff_or_final}"
    tp = results_dict[tp_str].clone()
    # fp = results_dict[fp_str]
    tn = results_dict[tn_str].clone()
    # fn = results_dict[fn_str]
    mask_sum_new = mask_sum.clone()
    for dim in sum_dims:
        mask_sum_new = mask_sum_new.sum(dim=dim)
        tp = tp.sum(dim=dim)
        tn = tn.sum(dim=dim)
    acc = (tp + tn) / mask_sum_new
    # acc = (results_dict[tp] + results_dict[tn]) / (results_dict[tp] + results_dict[tn] + results_dict[fp] + results_dict[fn])
    return acc

diff_or_final = "diff"

neuron_acts_approx_acc = get_accuracy(results_dict_neuron_acts_approx, mask_sum_neuron_acts_approx, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
neuron_acts_real_acc = get_accuracy(results_dict_neuron_acts_real, mask_sum_neuron_acts_real, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
base_acc = get_accuracy(results_dict_base, mask_sum_base, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
print(f"Neuron acts approx acc: {neuron_acts_approx_acc}")
print(f"Neuron acts real acc: {neuron_acts_real_acc}")
print(f"Base acc: {base_acc}")

diff_or_final = "final"

neuron_acts_approx_acc = get_accuracy(results_dict_neuron_acts_approx, mask_sum_neuron_acts_approx, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
neuron_acts_real_acc = get_accuracy(results_dict_neuron_acts_real, mask_sum_neuron_acts_real, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
base_acc = get_accuracy(results_dict_base, mask_sum_base, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
print(f"Neuron acts approx acc: {neuron_acts_approx_acc}")
print(f"Neuron acts real acc: {neuron_acts_real_acc}")
print(f"Base acc: {base_acc}")

diff_or_final = "normal"

neuron_acts_approx_acc = get_accuracy(results_dict_neuron_acts_approx, mask_normal_sum_neuron_acts_approx, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
neuron_acts_real_acc = get_accuracy(results_dict_neuron_acts_real, mask_normal_sum_neuron_acts_real, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
base_flipped_acc = get_accuracy(results_dict_base_flipped, mask_normal_sum_base_flipped, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
base_not_flipped_acc = get_accuracy(results_dict_base_not_flipped, mask_normal_sum_base_not_flipped, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
base_acc = get_accuracy(results_dict_base, mask_normal_sum_base, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
# base_acc = get_accuracy(results_dict_base, mask_sum_base, diff_or_final=diff_or_final, sum_dims=[3,2,1,0])
print(f"Neuron acts approx acc: {neuron_acts_approx_acc}")
print(f"Neuron acts real acc: {neuron_acts_real_acc}")
print(f"Base flipped acc: {base_flipped_acc}")
print(f"Base not flipped acc: {base_not_flipped_acc}")
print(f"Base acc: {base_acc}")
# print(f"Base acc: {base_acc}")

Neuron acts approx acc: 0.3584817945957184
Neuron acts real acc: 0.3361469507217407
Base acc: 0.3360779583454132
Neuron acts approx acc: 0.13220436871051788
Neuron acts real acc: 0.15360864996910095
Base acc: 0.14231732487678528
Neuron acts approx acc: 0.9205928444862366
Neuron acts real acc: 0.9291341304779053
Base flipped acc: 0.33145272731781006
Base not flipped acc: 0.6685472726821899
Base acc: 0.9235785007476807


In [66]:
# Eval sieht aus wie: Erste Layer decent, sonst kacke ... (naja ...), Layer 3 seems better again. idk lets see (It's good enough that I can put it is the thesis)
# Plot accuracy cool ...
acc = get_accuracy(results_dict_neuron_acts_real, mask_sum_neuron_acts_real, "final")
pos_size = 20
pos_start = 10
pos_end = pos_start + pos_size

# in acc replace nan with 0
acc[acc != acc] = 0

plot_boards_general(
    x_labels=[f'L{layer}' for layer in range(0, 8)],
    y_labels=[f'Pos {pos}' for pos in range(pos_start, pos_end)],
    boards = acc[:, pos_start:pos_end],
    color_range="not symmetric"
)
acc[1, 10]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000],
        [0.0000, 0.6000, 0.4000, 0.2353, 0.3333, 0.4444, 0.5000, 0.0000],
        [0.0000, 0.3333, 0.1695, 0.2388, 0.2481, 0.3115, 0.2500, 1.0000],
        [0.0000, 0.2258, 0.1912, 0.2105, 0.2218, 0.1842, 0.1481, 0.0000],
        [0.0000, 0.4444, 0.2519, 0.1596, 0.2568, 0.2175, 0.3333, 0.0000],
        [0.0000, 0.4667, 0.2692, 0.1329, 0.2517, 0.3697, 0.2500, 1.0000],
        [0.0000, 0.5000, 0.2222, 0.5882, 0.2667, 0.2000, 0.4545, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7500, 0.0000, 0.0000]],
       device='cuda:0')

In [58]:
test = t.ones((3, 3))

In [59]:
def get_f1_score_pos(results_dict, mask_sum, diff_or_final, pos):
    tp_str = f"TP_{diff_or_final}"
    fp_str = f"FP_{diff_or_final}"
    tn_str = f"TN_{diff_or_final}"
    fn_str = f"FN_{diff_or_final}"

    # Extract values for the specified position
    if pos is not None:
        tp = results_dict[tp_str][:, pos, :, :]
        fp = results_dict[fp_str][:, pos, :, :]
        fn = results_dict[fn_str][:, pos, :, :]
    else:
        tp = results_dict[tp_str].sum(dim=1)
        fp = results_dict[fp_str].sum(dim=1)
        fn = results_dict[fn_str].sum(dim=1)

    # Calculate for not_rim (inner region)
    tp_not_rim = tp[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    fp_not_rim = fp[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    fn_not_rim = fn[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    
    precision_not_rim = tp_not_rim / (tp_not_rim + fp_not_rim + 1e-10)
    recall_not_rim = tp_not_rim / (tp_not_rim + fn_not_rim + 1e-10)
    f1_not_rim = 2 * (precision_not_rim * recall_not_rim) / (precision_not_rim + recall_not_rim + 1e-10)

    # Calculate for rim (outer region)
    tp_rim = tp.sum(dim=(-2, -1)) - tp_not_rim
    fp_rim = fp.sum(dim=(-2, -1)) - fp_not_rim
    fn_rim = fn.sum(dim=(-2, -1)) - fn_not_rim
    
    precision_rim = tp_rim / (tp_rim + fp_rim + 1e-10)
    recall_rim = tp_rim / (tp_rim + fn_rim + 1e-10)
    f1_rim = 2 * (precision_rim * recall_rim) / (precision_rim + recall_rim + 1e-10)

    return f1_not_rim, f1_rim

def get_f1_score_layer(results_dict, mask_sum, diff_or_final, layer):
    tp_str = f"TP_{diff_or_final}"
    fp_str = f"FP_{diff_or_final}"
    tn_str = f"TN_{diff_or_final}"
    fn_str = f"FN_{diff_or_final}"

    # Extract values for the specified layer
    tp = results_dict[tp_str][layer, :, :, :]
    fp = results_dict[fp_str][layer, :, :, :]
    fn = results_dict[fn_str][layer, :, :, :]

    # Calculate for not_rim (inner region)
    tp_not_rim = tp[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    fp_not_rim = fp[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    fn_not_rim = fn[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    
    precision_not_rim = tp_not_rim / (tp_not_rim + fp_not_rim + 1e-10)
    recall_not_rim = tp_not_rim / (tp_not_rim + fn_not_rim + 1e-10)
    f1_not_rim = 2 * (precision_not_rim * recall_not_rim) / (precision_not_rim + recall_not_rim + 1e-10)

    # Calculate for rim (outer region)
    tp_rim = tp.sum(dim=(-2, -1)) - tp_not_rim
    fp_rim = fp.sum(dim=(-2, -1)) - fp_not_rim
    fn_rim = fn.sum(dim=(-2, -1)) - fn_not_rim
    
    precision_rim = tp_rim / (tp_rim + fp_rim + 1e-10)
    recall_rim = tp_rim / (tp_rim + fn_rim + 1e-10)
    f1_rim = 2 * (precision_rim * recall_rim) / (precision_rim + recall_rim + 1e-10)

    return f1_not_rim, f1_rim

In [60]:
def get_accuracy_pos(results_dict, mask_sum, diff_or_final, pos):
    tp_str = f"TP_{diff_or_final}"
    fp_str = f"FP_{diff_or_final}"
    tn_str = f"TN_{diff_or_final}"
    fn_str = f"FN_{diff_or_final}"

    if pos is not None:
        correct = (results_dict[tp_str] + results_dict[tn_str])[:, pos, :, :]
    else:
        correct = (results_dict[tp_str] + results_dict[tn_str]).sum(dim=1)
    correct_not_rim = correct[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    correct_rim = correct.sum(dim=(-2, -1)) - correct_not_rim
    if pos is not None:
        not_blank = mask_sum[:, pos, :, :]
    else:
        not_blank = mask_sum.sum(dim=1)
    not_blank_not_rim = not_blank[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    not_blank_rim = not_blank.sum(dim=(-2, -1)) - not_blank_not_rim
    accuracy_not_rim = correct_not_rim / not_blank_not_rim
    accuracy_rim = correct_rim / not_blank_rim
    return accuracy_not_rim, accuracy_rim

def get_accuracy_layer(results_dict, mask_sum, diff_or_final, layer):
    tp_str = f"TP_{diff_or_final}"
    fp_str = f"FP_{diff_or_final}"
    tn_str = f"TN_{diff_or_final}"
    fn_str = f"FN_{diff_or_final}"

    correct = (results_dict[tp_str] + results_dict[tn_str])[layer, :, :, :]
    correct_not_rim = correct[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    correct_rim = correct.sum(dim=(-2, -1)) - correct_not_rim
    not_blank = mask_sum[layer, :, :, :]
    not_blank_not_rim = not_blank[:, 1:-1, 1:-1].sum(dim=(-2, -1))
    not_blank_rim = not_blank.sum(dim=(-2, -1)) - not_blank_not_rim
    accuracy_not_rim = correct_not_rim / not_blank_not_rim
    accuracy_rim = correct_rim / not_blank_rim
    return accuracy_not_rim, accuracy_rim

'''def get_f1_score_pos(results_dict, mask_sum, diff_or_final, pos):
    pass

def get_f1_score_layer(results_dict, mask_sum, diff_or_final, layer):
    pass'''

layer = 5
pos = None
diff_or_final = "final"

accuracy_pos_not_rim_neuron_acts_approx, accuracy_pos_rim_neuron_acts_approx = get_accuracy_pos(results_dict_neuron_acts_approx, mask_sum_neuron_acts_approx, diff_or_final, pos)
accuracy_pos_not_rim_neuron_acts_real, accuracy_pos_rim_neuron_acts_real = get_accuracy_pos(results_dict_neuron_acts_real, mask_sum_neuron_acts_real, diff_or_final, pos)
accuracy_pos_not_rim_base_flipped, accuracy_pos_rim_base_flipped = get_accuracy_pos(results_dict_base_flipped, mask_sum_base_flipped, diff_or_final, pos)
accuracy_pos_not_rim_base_not_flipped, accuracy_pos_rim_base_not_flipped = get_accuracy_pos(results_dict_base_not_flipped, mask_sum_base_not_flipped, diff_or_final, pos)
accuracy_pos_not_rim_base, accuracy_pos_rim_base = get_accuracy_pos(results_dict_base, mask_sum_base, diff_or_final, pos)

fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approx Neuron Acts (Middle)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approx Neuron Acts (Rim)', line=dict(dash='dash', color='blue')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts(Middle)', line=dict(color='red')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts (Rim)', line=dict(dash='dash', color='red')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base_flipped.cpu(), mode='lines+markers', name='Baseline Flipped (Middle)', line=dict(color='green')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base_flipped.cpu(), mode='lines+markers', name='Baseline Flipped (Rim)', line=dict(dash='dash', color='green')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base_not_flipped.cpu(), mode='lines+markers', name='Baseline Not Flipped(Middle)', line=dict(color='purple')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base_not_flipped.cpu(), mode='lines+markers', name='Baseline Not Flipped (Rim)', line=dict(dash='dash', color='purple')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base.cpu(), mode='lines+markers', name='Baseline (Middle)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base.cpu(), mode='lines+markers', name='Baseline (Rim)', line=dict(dash='dash', color='green')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base.cpu(), mode='lines+markers', name='Baseline (Rim)', line=dict(dash='dash', color='green')))

title = f'Accuracy of Flipping Circuit over the layers'
fig.update_layout(
    title=title,
    xaxis_title='Layer',
    yaxis_title='Accuracy',
)
fig.show()
save_plotly(fig, title)
diff_or_final = "diff"

accuracy_pos_not_rim_neuron_acts_approx, accuracy_pos_rim_neuron_acts_approx = get_accuracy_pos(results_dict_neuron_acts_approx, mask_sum_neuron_acts_approx, diff_or_final, pos)
accuracy_pos_not_rim_neuron_acts_real, accuracy_pos_rim_neuron_acts_real = get_accuracy_pos(results_dict_neuron_acts_real, mask_sum_neuron_acts_real, diff_or_final, pos)
accuracy_pos_not_rim_base_flipped, accuracy_pos_rim_base_flipped = get_accuracy_pos(results_dict_base_flipped, mask_sum_base_flipped, diff_or_final, pos)
accuracy_pos_not_rim_base_not_flipped, accuracy_pos_rim_base_not_flipped = get_accuracy_pos(results_dict_base_not_flipped, mask_sum_base_not_flipped, diff_or_final, pos)

fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approx Neuron Acts (NMiddle', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approx Neuron Acts (Rim)', line=dict(dash='dash', color='blue')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts(Middle)', line=dict(color='red')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts (Rim)', line=dict(dash='dash', color='red')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base_flipped.cpu(), mode='lines+markers', name='Baseline Flipped (Middle)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base_flipped.cpu(), mode='lines+markers', name='Baseline Flipped (Rim)', line=dict(dash='dash', color='green')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base_not_flipped.cpu(), mode='lines+markers', name='Baseline Not Flipped(Middle)', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base_not_flipped.cpu(), mode='lines+markers', name='Baseline Not Flipped (Rim)', line=dict(dash='dash', color='purple')))

title = f'Accuracy of MLP approximation over the layers (Easy Evaluation)'
fig.update_layout(
    title=title,
    xaxis_title='Layer',
    yaxis_title='Accuracy',
)
fig.show()

'''accuracy_pos_not_rim_neuron_acts_approx, accuracy_pos_rim_neuron_acts_approx = get_accuracy_layer(results_dict_neuron_acts_approx, mask_sum_neuron_acts_approx, diff_or_final, layer)
accuracy_pos_not_rim_neuron_acts_real, accuracy_pos_rim_neuron_acts_real = get_accuracy_layer(results_dict_neuron_acts_real, mask_sum_neuron_acts_real, diff_or_final, layer)
accuracy_pos_not_rim_base, accuracy_pos_rim_base = get_accuracy_layer(results_dict_base, mask_sum_base, diff_or_final, layer)

# plot a line graph with plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_pos_not_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approximated Neuron Acts (Not Rim)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_pos_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approximated Neuron Acts (Rim)', line=dict(dash='dash', color='blue')))
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_pos_not_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts(Not Rim)', line=dict(color='red')))
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_pos_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts (Rim)', line=dict(dash='dash', color='red')))
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_pos_not_rim_base.cpu(), mode='lines+markers', name='Baseline (Not Rim)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=[f'{pos}' for pos in range(0, 59)], y=accuracy_pos_rim_base.cpu(), mode='lines+markers', name='Baseline (Rim)', line=dict(dash='dash', color='green')))
title = f'Accuracy of MLP approximation at layer {layer} over the positions (Easy Evaluation)'
fig.update_layout(
    title=title,
    xaxis_title='Layer',
    yaxis_title='Accuracy',
)
fig.show()
# Baseline and Neuron Acts Real and not real
# save_plotly(fig, title)'''

pos = None
diff_or_final = "normal"

accuracy_pos_not_rim_neuron_acts_approx, accuracy_pos_rim_neuron_acts_approx = get_f1_score_pos(results_dict_neuron_acts_approx, mask_normal_sum_neuron_acts_approx, diff_or_final, pos)
accuracy_pos_not_rim_neuron_acts_real, accuracy_pos_rim_neuron_acts_real = get_f1_score_pos(results_dict_neuron_acts_real, mask_normal_sum_neuron_acts_real, diff_or_final, pos)
# accuracy_pos_not_rim_base_flipped, accuracy_pos_rim_base_flipped = get_accuracy_pos(results_dict_base_flipped, mask_normal_sum_base_flipped, diff_or_final, pos)
# accuracy_pos_not_rim_base_not_flipped, accuracy_pos_rim_base_not_flipped = get_accuracy_pos(results_dict_base_not_flipped, mask_normal_sum_base_not_flipped, diff_or_final, pos)
# accuracy_pos_not_rim_base, accuracy_pos_rim_base = get_accuracy_pos(results_dict_base, mask_normal_sum_base, diff_or_final, pos)

fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approximated Neuron Acts (Not Rim)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_neuron_acts_approx.cpu(), mode='lines+markers', name='Approximated Neuron Acts (Rim)', line=dict(dash='dash', color='blue')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts(Not Rim)', line=dict(color='red')))
fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_neuron_acts_real.cpu(), mode='lines+markers', name='Real Neuron Acts (Rim)', line=dict(dash='dash', color='red')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base_flipped.cpu(), mode='lines+markers', name='Baseline Flipped (Not Rim)', line=dict(color='green')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base_flipped.cpu(), mode='lines+markers', name='Baseline Flipped(Rim)', line=dict(dash='dash', color='green')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base_not_flipped.cpu(), mode='lines+markers', name='Baseline Not Flipped (Not Rim)', line=dict(color='purple')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base_not_flipped.cpu(), mode='lines+markers', name='Baseline Not Flipped (Rim)', line=dict(dash='dash', color='purple')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_not_rim_base.cpu(), mode='lines+markers', name='Baseline (Not Rim)', line=dict(color='orange')))
# fig.add_trace(go.Scatter(x=[f'{layer}' for layer in range(0, 8)], y=accuracy_pos_rim_base.cpu(), mode='lines+markers', name='Baseline (Rim)', line=dict(dash='dash', color='orange')))


title = f'Accuracy of MLP approximation over the layers (Hard Evaluation)'
fig.update_layout(
    title=title,
    xaxis_title='Layer',
    yaxis_title='Accuracy',
)
fig.show()

In [20]:
'''
2048
real true
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.8000, 0.8824, 0.8889, 0.6667, 0.0000, 0.0000],
        [0.0000, 0.6111, 0.7373, 0.7970, 0.5963, 0.7213, 0.6250, 1.0000],
        [0.0000, 0.8710, 0.6928, 0.6203, 0.7055, 0.7138, 0.5926, 0.0000],
        [0.0000, 0.7778, 0.7519, 0.7472, 0.7262, 0.7288, 0.7222, 0.0000],
        [1.0000, 0.5333, 0.7788, 0.7902, 0.7448, 0.8151, 0.7500, 0.0000],
        [0.0000, 0.0000, 0.5556, 0.5882, 0.7333, 0.6000, 0.6364, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000]],
       device='cuda:0')
real false
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.8000, 0.9375, 0.9000, 0.6667, 0.0000, 0.0000],
        [0.0000, 0.6111, 0.7458, 0.8542, 0.6803, 0.7077, 0.6111, 1.0000],
        [0.0000, 0.8889, 0.8137, 0.7472, 0.8006, 0.7390, 0.5909, 0.0000],
        [0.0000, 0.7368, 0.7975, 0.8186, 0.7980, 0.7845, 0.6250, 0.0000],
        [1.0000, 0.5000, 0.7895, 0.7938, 0.8108, 0.7961, 0.7500, 0.0000],
        [0.0000, 0.5000, 0.4286, 0.5882, 0.7333, 0.7000, 0.7273, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000]],
       device='cuda:0')
20
real true
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8333, 0.8000, 0.8750, 0.8889, 0.6667, 0.0000, 0.0000],
        [0.0000, 0.6111, 0.7436, 0.8153, 0.6015, 0.7213, 0.5882, 1.0000],
        [0.0000, 0.8846, 0.7077, 0.6241, 0.7073, 0.7526, 0.6250, 0.0000],
        [0.0000, 0.7778, 0.7588, 0.8012, 0.7283, 0.7485, 0.6667, 0.0000],
        [1.0000, 0.5333, 0.8172, 0.8000, 0.7535, 0.8261, 0.7273, 0.0000],
        [0.0000, 0.0000, 0.4286, 0.5882, 0.6667, 0.6000, 0.6364, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000]],
       device='cuda:0')
real false
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.8000, 0.9375, 0.8947, 0.6250, 0.0000, 0.0000],
        [0.0000, 0.6667, 0.7479, 0.8556, 0.6748, 0.7031, 0.6111, 1.0000],
        [0.0000, 0.8889, 0.8161, 0.7424, 0.8004, 0.7415, 0.5909, 0.0000],
        [0.0000, 0.7895, 0.7967, 0.8291, 0.7951, 0.7938, 0.6250, 0.0000],
        [1.0000, 0.5000, 0.7957, 0.7946, 0.8086, 0.7885, 0.7500, 0.0000],
        [0.0000, 0.5000, 0.4286, 0.5882, 0.7333, 0.7000, 0.6364, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000]],
       device='cuda:0')
'''

"\n2048\nreal true\ntensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],\n        [0.0000, 1.0000, 0.8000, 0.8824, 0.8889, 0.6667, 0.0000, 0.0000],\n        [0.0000, 0.6111, 0.7373, 0.7970, 0.5963, 0.7213, 0.6250, 1.0000],\n        [0.0000, 0.8710, 0.6928, 0.6203, 0.7055, 0.7138, 0.5926, 0.0000],\n        [0.0000, 0.7778, 0.7519, 0.7472, 0.7262, 0.7288, 0.7222, 0.0000],\n        [1.0000, 0.5333, 0.7788, 0.7902, 0.7448, 0.8151, 0.7500, 0.0000],\n        [0.0000, 0.0000, 0.5556, 0.5882, 0.7333, 0.6000, 0.6364, 0.0000],\n        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.0000, 0.0000]],\n       device='cuda:0')\nreal false\ntensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],\n        [0.0000, 1.0000, 0.8000, 0.9375, 0.9000, 0.6667, 0.0000, 0.0000],\n        [0.0000, 0.6111, 0.7458, 0.8542, 0.6803, 0.7077, 0.6111, 1.0000],\n        [0.0000, 0.8889, 0.8137, 0.7472, 0.8006, 0.7390, 0.5909, 0.0000],\n        [0.0000, 0.7368, 0.7975, 0.8186, 0

### Mean Neuron Activation Difference for each Rule
- avg number of neurons above thresholds (0.17 / 1 etc.)
- Viele Regeln haben 0 Neurons (In bestimmter Layer ...)
- Gibt aber auch ordentlich viele Outlier

In [21]:
'''fig = go.Figure()
fig.add_trace(go.Histogram(x=neuron_acts.cpu()))
fig.update_layout(title=f"Rule {rule}, Layer {layer}")
fig.show()'''

'fig = go.Figure()\nfig.add_trace(go.Histogram(x=neuron_acts.cpu()))\nfig.update_layout(title=f"Rule {rule}, Layer {layer}")\nfig.show()'

In [22]:
neuron_acts_dict_neuron_acts_real.keys()

dict_keys([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (10, 0), (10, 2), (10, 5), (10, 6), (10, 7), (11, 0), (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (12, 6), (12, 7), (13, 0), (13, 1), (13, 2), (13, 3), (13, 4), (13, 5), (13, 6), (13, 7), (14, 0), (14, 1), (14, 2), (14, 3), (14, 4), (14, 5), (14, 6), (14, 7), (15, 0), (15, 1), (15, 2

In [23]:
neuron_acts = neuron_acts_dict_neuron_acts_real[1, 1]
neuron_acts[neuron_acts > 0.17]

tensor([0.1793, 0.2415, 0.3065, 0.1968, 0.2004, 0.2415, 0.2174, 0.1778],
       device='cuda:0')

In [24]:
threshold = 0.17

for layer_main in range(0, 8):
    neuron_acts = []
    for rule in range(0, rule_max):
        this_rule = []
        for layer in range(layer_main, layer_main + 1):
            if (rule, layer) not in neuron_acts_dict_neuron_acts_approx:
                this_rule.append(t.zeros(2048, device=device))
                continue
            this_rule.append(neuron_acts_dict_neuron_acts_real[(rule, layer)])
        this_rule = t.cat(this_rule, dim=0)
        neuron_acts.append(this_rule)
    neuron_acts = t.stack(neuron_acts, dim=0)
    num_neurons_above_threshold = (neuron_acts > threshold).sum(dim=1)
    fig = go.Figure()
    # fig.add_trace(go.Histogram(x=num_neurons_above_threshold.cpu(), xbins_size=1))
    fig.add_trace(go.Histogram(x=num_neurons_above_threshold.cpu()))
    fig.update_layout(title=f"layer {layer_main}")
    fig.show()

In [52]:
# Use Plotly's default sequential colors - these are vibrant and professionally designed
colors = [
    '#636EFA', '#EF553B', '#00CC96', '#AB63FA', 
    '#FFA15A', '#19D3F3', '#FF6692', '#B6E880'
]

# Create a single figure
fig = go.Figure()

# Add histograms for each layer with consistent binning
for layer_main in range(0, 8):
    neuron_acts = []
    for rule in range(0, rule_max):
        this_rule = []
        for layer in range(layer_main, layer_main + 1):
            if (rule, layer) not in neuron_acts_dict_neuron_acts_approx:
                this_rule.append(t.zeros(2048, device=device))
                continue
            this_rule.append(neuron_acts_dict_neuron_acts_real[(rule, layer)])
        this_rule = t.cat(this_rule, dim=0)
        neuron_acts.append(this_rule)
    neuron_acts = t.stack(neuron_acts, dim=0)
    num_neurons_above_threshold = (neuron_acts > threshold).sum(dim=1)
    
    fig.add_trace(go.Histogram(
        x=num_neurons_above_threshold.cpu(),
        name=f'Layer {layer_main}',
        marker_color=colors[layer_main],
        opacity=1.0,  # Full opacity for vibrant colors
        xbins_size=3,
        histfunc="avg",
    ))

# Update layout to make histograms stacked and use log scale
fig.update_layout(
    barmode='stack',
    title="Number of Neurons Above Threshold Across Layers",
    xaxis_title="Number of Neurons",
    yaxis_title="Number of Rules",
    # xaxis_type="log",
    yaxis_type="log",  # Set y-axis to log scale
    # xaxis_type="log",
    showlegend=True,
    template='plotly_white'
)

fig.show()
save_plotly(fig, "Number of Rules with a certain number of neurons with mean activation difference > 0.17")

In [30]:
# Use Plotly's default vibrant colors
colors = [
    '#636EFA', '#EF553B', '#00CC96', '#AB63FA', 
    '#FFA15A', '#19D3F3', '#FF6692', '#B6E880'
]

# Create arrays to store data for each layer
data_by_rule = []
for rule in range(rule_max):
    neurons_by_layer = []
    for layer in range(8):
        if (rule, layer) not in neuron_acts_dict_neuron_acts_approx:
            neurons_by_layer.append(0)
            continue
        neuron_acts = neuron_acts_dict_neuron_acts_real[(rule, layer)]
        num_neurons = (neuron_acts > threshold).sum().item()
        neurons_by_layer.append(num_neurons)
    data_by_rule.append(neurons_by_layer)

# Convert to numpy for easier manipulation
data_array = np.array(data_by_rule)

# Calculate total neurons per rule for sorting
total_neurons_per_rule = data_array.sum(axis=1)
sort_indices = np.argsort(total_neurons_per_rule)[::-1]  # Sort in descending order

# Create figure
fig = go.Figure()

# Add bars for each layer
for layer in range(8):
    fig.add_trace(go.Bar(
        name=f'Layer {layer}',
        x=np.arange(len(sort_indices)),  # Use consecutive integers for x-axis
        y=data_array[:, layer][sort_indices],  # Get data for this layer, sorted
        marker_color=colors[layer],
        marker=dict(line=dict(width=0)),  # Remove bar borders
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    bargap=0,  # Remove gaps between bars
    title="Number of Neurons Above Threshold by Rule and Layer",
    xaxis_title="Rule Index (sorted by total neurons)",
    yaxis_title="Number of Neurons",
    yaxis_type="log",
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [43]:
# Use Plotly's default vibrant colors
colors = [
    '#636EFA', '#EF553B', '#00CC96', '#AB63FA', 
    '#FFA15A', '#19D3F3', '#FF6692', '#B6E880'
]

layer_max = 8

# First, collect all data and calculate totals for sorting
data_by_rule = []
for rule in range(rule_max):
    neurons_by_layer = []
    for layer in range(layer_max):
        if (rule, layer) not in neuron_acts_dict_neuron_acts_real:
            neurons_by_layer.append(0)
            continue
        acts = neuron_acts_dict_neuron_acts_real[(rule, layer)]
        num_neurons = (acts > threshold).sum().item()
        neurons_by_layer.append(num_neurons)
    data_by_rule.append(neurons_by_layer)

# Convert to numpy for sorting
data_array = np.array(data_by_rule)
total_neurons_per_rule = data_array.sum(axis=1)
sort_indices = np.argsort(total_neurons_per_rule)[::-1]  # Sort in descending order

fig = go.Figure()

# Add histogram traces for each layer using sorted data
for layer in range(layer_max):
    fig.add_trace(go.Histogram(
        x=np.arange(len(sort_indices)),  # Use consecutive integers for x-axis
        y=data_array[:, layer][sort_indices],  # Use sorted data
        name=f'Layer {layer}',
        marker_color=colors[layer],
        histfunc='avg',
        # nbinsx=rule_max,
        opacity=1.0,
        # autobinx=False,
    ))

# Update layout
fig.update_layout(
    barmode='stack',
    bargap=0,
    title="Number of Neurons Above Threshold by Rule and Layer (Sorted)",
    xaxis_title="Rule Index (sorted by total neurons)",
    yaxis_title="Number of Neurons",
    yaxis_type="log",
    showlegend=True,
    template='plotly_white'
)

fig.show()
save_plotly(fig, "Number of Neurons Above Threshold by Rule and Layer (Sorted)")